In [1]:
#imports
import os, shutil, fnmatch, time, tkinter
from tkinter.filedialog import askdirectory
from pathlib import Path

In [2]:
#prompt user for the file directory. Will open as a popup window named "tk"
tk_root = tkinter.Tk()
path_input = askdirectory(title='Select Folder')  # shows dialog box and return the path
print(path_input)
tk_root.destroy()

D:/20230922_NHDFneo_P4_Fluor-FN-Col1/BF/Sorted/tissue_ai_w1Brightfield


In [3]:
stage_pos_chars = []
for file in os.listdir(path_input):
    if file.endswith('.nd'):
        basename = file.split('.',1)[0]
        print(basename)
        with open(path_input + '/' +file, 'r') as nd_file:
            for l_no, line in enumerate(nd_file):
                if '"NStagePositions"' in line:
                    stage_pos_nd = [int(s) for s in line.split() if s.isdigit()][-1]
                if '"NTimePoints"' in line:
                    timepoints_nd = [int(s) for s in line.split() if s.isdigit()][-1]
                if '"Stage' in line:
                    spc = line.split(', "')[0].split('Stage')[-1]
                    #stage_pos_chars.append(spc[0:len(spc)-2])
                    print(spc)

print(stage_pos_nd)

tissue_ai_w1Brightfield
1"
2"
3"
4"
5"
6"
7"
8"
9"
10"
11"
12"
13"
14"
15"
16"
17"
18"
19"
20"
21"
22"
23"
24"
25"
26"
27"
28"
29"
30"
31"
32"
33"
34"
35"
36"
37"
38"
39"
40"
41"
42"
42


In [5]:
for file in os.listdir(path_input):
    if os.path.isdir(path_input + '/' + file):
        #rename file for correct sort syntax
                folder = file.split('s', 1)[-1]

                for num in range(2-len(folder)):
                    folder = '0'+folder
                newfile = 's'+folder
                os.rename(path_input + '/' + file, path_input + '/' + newfile)


PermissionError: [WinError 5] Access is denied: 'D:/20230922_NHDFneo_P4_Fluor-FN-Col1/BF/Sorted/tissue_ai_w1Brightfield/s1' -> 'D:/20230922_NHDFneo_P4_Fluor-FN-Col1/BF/Sorted/tissue_ai_w1Brightfield/s01'

In [16]:
import pandas as pd
list_pos= []
tlist = [T*0.5 for T in range(1,timepoints_nd+1)]
wa_df = pd.DataFrame({'Frame': range(1,timepoints_nd+1),
                      'Time':  tlist})
for file in os.listdir(path_input):
    if os.path.isdir(path_input+'/'+file):

        list_pos.append(file)
        wa_df[file] = pd.read_table(path_input+'/'+file+'/segment_ph1/is_closed_vs_frame.txt',header = None)

#header1 = wa_df.columns
#header2 = ['t','Hours']+ stage_pos_chars
#wa_df.columns = [header1, header2]
display(wa_df)

xls_name = '20230921'
#wa_df.to_excel(r'D:/20230922_NHDFneo_P4_Fluor-FN-Col1/BF/Sorted/Wound_area_code_output.xlsx', sheet_name="isbroken")

def append_to_excel(fpath, df, sheet_name):
    with pd.ExcelWriter(fpath, mode="a") as f:
        df.to_excel(f, sheet_name=sheet_name)

append_to_excel('D:/20230922_NHDFneo_P4_Fluor-FN-Col1/BF/Sorted/Wound_area_code_output.xlsx',wa_df, "isclosed")# "wound_area")


,Frame,Time,s01,s02,s03,s04,s05,s06,s07,s08,...,s33,s34,s35,s36,s37,s38,s39,s40,s41,s42
0,1,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1.5,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,2.5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,3.5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,8,4.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
8,9,4.5,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
9,10,5.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
